In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import datetime

from shapely.geometry import Point, shape 
import geopandas as gpd
import folium
from folium import plugins
import branca
from branca.colormap import linear
from folium.plugins import TimeSliderChoropleth

In [2]:
listing_df = pd.read_csv('listings.csv', parse_dates=['last_review'])
nbh_geo_df = gpd.read_file('neighbourhoods.geojson', driver='GeoJSON')

listing_df = listing_df[['id', 'room_type', 'neighbourhood', 'latitude', 'longitude', 'price']]
nbh_geo_df = nbh_geo_df[['neighbourhood', 'geometry']]

In [3]:
listing_df.head(2)

,id,room_type,neighbourhood,latitude,longitude,price
0,3109,Entire home/apt,Observatoire,48.83349,2.31852,60
1,5396,Entire home/apt,Hôtel-de-Ville,48.85100,2.35869,115


In [4]:
nbh_geo_df.head(2)

,neighbourhood,geometry
0,Batignolles-Monceau,"(POLYGON ((2.295167 48.873955, 2.29504 48.8737..."
1,Palais-Bourbon,"(POLYGON ((2.320903 48.863057, 2.320941 48.863..."


In [5]:
locs_geometry = [Point(xy) for xy in zip(listing_df.longitude, listing_df.latitude)]
crs = {'init': 'epsg:4326'} # Coordinate Reference Systems, "epsg:4326" is a common projection of WGS84 Latitude/Longitude
locs_gdf = gpd.GeoDataFrame(listing_df, crs=crs, geometry=locs_geometry)

## Display all locations with points on the map
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Popups.ipynb

In [6]:
locs_map = folium.Map(location=[48.856614, 2.3522219], zoom_start = 13, tiles='cartodbpositron')

feature_ea = folium.FeatureGroup(name='Entire home/apt')
feature_pr = folium.FeatureGroup(name='Private room')
feature_sr = folium.FeatureGroup(name='Shared room')

for i, v in locs_gdf.iterrows():
    popup = """
    Location id : <b>%s</b><br>
    Room type : <b>%s</b><br>
    Neighbourhood : <b>%s</b><br>
    Price : <b>%d</b><br>
    """ % (v['id'], v['room_type'], v['neighbourhood'], v['price'])
    
    if v['room_type'] == 'Entire home/apt':
        folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=1,
                            tooltip=popup,
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_ea)
    elif v['room_type'] == 'Private room':
        folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=1,
                            tooltip=popup,
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_pr)
    elif v['room_type'] == 'Shared room':
        folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=1,
                            tooltip=popup,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_sr)

feature_ea.add_to(locs_map)
feature_pr.add_to(locs_map)
feature_sr.add_to(locs_map)
folium.LayerControl(collapsed=False).add_to(locs_map)

In [7]:
locs_map.save('locs_map.html')

## Display all locations with clusters
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/MarkerCluster.ipynb

In [8]:
cluster_map = folium.Map(location=[48.856614, 2.3522219], zoom_start = 13, tiles='cartodbpositron')

marker_cluster = plugins.MarkerCluster().add_to(cluster_map)

for i, v in locs_gdf.iterrows():
    popup = """
    Location id : <b>%s</b><br>
    Room type : <b>%s</b><br>
    Neighbourhood : <b>%s</b><br>
    Price : <b>%d</b><br>
    """ % (v['id'], v['room_type'], v['neighbourhood'], v['price'])
    
    if v['room_type'] == 'Entire home/apt':
        folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=3,
                            tooltip=popup,
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(marker_cluster)
    elif v['room_type'] == 'Private room':
        folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=3,
                            tooltip=popup,
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(marker_cluster)
    elif v['room_type'] == 'Shared room':
        folium.CircleMarker(location=[v['latitude'], v['longitude']],
                            radius=3,
                            tooltip=popup,
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(marker_cluster)

In [9]:
cluster_map.save('cluster_map.html')

## Paint areas with different colors
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Colormaps.ipynb

In [10]:
nbh_count_df = listing_df.groupby('neighbourhood')['id'].nunique().reset_index()
nbh_count_df.rename(columns={'id':'nb'}, inplace=True)
nbh_geo_count_df = pd.merge(nbh_geo_df, nbh_count_df, on='neighbourhood')
nbh_geo_count_df['QP'] = nbh_geo_count_df['nb'] / nbh_geo_count_df['nb'].sum()
nbh_geo_count_df['QP_str'] = nbh_geo_count_df['QP'].apply(lambda x : str(round(x*100, 1)) + '%')

In [11]:
nbh_count_colormap = linear.YlGnBu_09.scale(min(nbh_count_df['nb']), max(nbh_count_df['nb']))

nbh_locs_map = folium.Map(location=[48.856614, 2.3522219], zoom_start = 12, tiles='cartodbpositron')

style_function = lambda x: {
    'fillColor': nbh_count_colormap(x['properties']['nb']),
    'color': 'black',
    'weight': 1.5,
    'fillOpacity': 0.7
}


folium.GeoJson(
    nbh_geo_count_df,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['neighbourhood', 'nb', 'QP_str'],
        aliases=['Neighbourhood', 'Location amount', 'Quote-part'],
        localize=True
    )
).add_to(nbh_locs_map)


nbh_count_colormap.add_to(nbh_locs_map)
nbh_count_colormap.caption = 'Airbnb location amount'
nbh_count_colormap.add_to(nbh_locs_map)

In [12]:
nbh_locs_map.save('nbh_locs_map.html')

## Add labels on the map
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/CustomPanes.ipynb

In [13]:
nbh_locs_label_map = folium.Map(location=[48.856614, 2.3522219], zoom_start = 12, tiles='cartodbpositron')

style_function = lambda x: {
    'fillColor': nbh_count_colormap(x['properties']['nb']),
    'color': 'black',
    'weight': 1,
    'fillOpacity': 0.7
}


folium.GeoJson(
    nbh_geo_count_df,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['neighbourhood', 'nb', 'QP_str'],
        aliases=['Neighbourhood', 'Location amount', 'Quote-part'],
        localize=True
    )
).add_to(nbh_locs_label_map)

folium.map.CustomPane('labels').add_to(nbh_locs_label_map)
folium.TileLayer('CartoDBPositronOnlyLabels', pane='labels').add_to(nbh_locs_label_map)

In [14]:
nbh_locs_label_map.save('nbh_locs_label_map.html')

## Add polygon borders
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb

In [15]:
dept_geo = gpd.read_file('departements.geojson', driver='GeoJSON')
departments = {'75', '77', '78', '91', '92', '93', '94', '95'}
dept_geo = dept_geo[dept_geo['code'].isin(departments)]

In [16]:
polygon_map = folium.Map(location=[48.716614, 2.3522219], zoom_start = 9, tiles='cartodbpositron')

style_function = lambda x: {
    'fillColor': nbh_count_colormap(x['properties']['nb']),
    'color': 'black',
    'weight': 1.5,
    'fillOpacity': 0.7
}


folium.GeoJson(
    nbh_geo_count_df,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['neighbourhood', 'nb', 'QP_str'],
        aliases=['Neighbourhood', 'Location amount', 'Quote-part'],
        localize=True
    )
).add_to(polygon_map)

folium.GeoJson(
    dept_geo,
    style_function = lambda x: {
        'color': 'black',
        'weight': 2.5,
        'fillOpacity': 0
    },
    name='Departement').add_to(polygon_map)

nbh_count_colormap.add_to(polygon_map)
nbh_count_colormap.caption = 'Airbnb location amount'
nbh_count_colormap.add_to(polygon_map)


In [17]:
polygon_map.save('polygon_map.html')

## Show changes in terms of timing with heatmap
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/HeatMapWithTime.ipynb

In [ ]:
date_index = [d.strftime('%Y-%m-%d') for d in listing_summary_history.date_of_data.unique()][::-1]

In [ ]:
locs_history_map = folium.Map(location=[48.856614, 2.3522219], zoom_start = 13, tiles='cartodbpositron')

hm = plugins.HeatMapWithTime(
    locs_yearly,
    index=date_index,
    auto_play=True,
    radius=4,
    max_opacity=0.3
)

hm.add_to(locs_history_map)

## Show changes in terms of timing with choropleth
https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb

In [ ]:
nbh_locsNb_history = listing_summary_history.groupby(['date_of_data',
                                                      'neighbourhood'])['id'].nunique().reset_index()
nbh_locsNb_history.rename(columns={'id':'nb'}, inplace=True)
nbh_locsNb_history.sort_values(['neighbourhood', 'date_of_data'], inplace=True)
nbh_locsNb_history.reset_index(drop=True, inplace=True)

In [ ]:
nbh_geo_sorted_df = nbh_geo_df.sort_values('neighbourhood').reset_index(drop=True)

datetime_index = pd.DatetimeIndex(nbh_locsNb_history.date_of_data.unique())
dt_index_epochs = datetime_index.astype(int) // 10**9
dt_index = np.array(dt_index_epochs).astype('U10')

styledata = {}
s = 0
e = 44
for i, v in nbh_geo_sorted_190709.iterrows():
    df = pd.DataFrame(
        {'color': np.array(nbh_locsNb_history.nb[s:e]),
         'opacity': np.array([1] * 44)},
        index=dt_index
    )
    styledata[i] = df
    s += 44
    e += 44

max_color = max(nbh_locsNb_history['nb'])
min_color = min(nbh_locsNb_history['nb'])
max_opacity, min_opacity = 1, 1

cmap = linear.YlGnBu_09.scale(min_color, max_color)

def norm(x):
    return (x - x.min()) / (x.max() - x.min())

for i, data in styledata.items():
    data['color'] = data['color'].map(cmap)
    data['opacity'] = 1

styledict = {
    str(nbh): data.to_dict(orient='index') for nbh, data in styledata.items()
}

In [ ]:
nbh_locs_history_map = folium.Map(location=[48.856614, 2.3522219], zoom_start = 12, tiles='cartodbpositron')

TimeSliderChoropleth(
    nbh_geo_sorted_190709.to_json(),
    styledict=styledict
).add_to(nbh_locs_history_map)

nbh_locs_history_map